In this lesson, I want to learn about how to dealing with rate limit in hugging face
1. I want to test the error message that I will get from hf

** result try to make request untill reach the rate limit is not work because It have a lot of limit
** next experiment -> try to simulate the rate limit

In [ ]:
%pip install python-dotenv

In [ ]:
import requests
import time
from dotenv import load_dotenv
import os

# load env variable from .env file in the same directory
load_dotenv() #use from jupyter remote kernel

# get token
hf_token = os.getenv("HUGGINGFACE_TOKEN")

if not hf_token:
    print("❌ ERROR: Token not found! Set HUGGINGFACE_TOKEN on your VPS")
else:
    print("✅ Token found")

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(token=hf_token) # we don't specific model name so it use default model
result = client.text_classification("I love this!!")
print(result)

In [ ]:
# provoke to see the error
# this method not work because it's has a lot of rate limit per hour
for i in range(1000):
    try:
        prompt = f"this is number {i}"
        response = client.text_classification(prompt)
        print(f"Request {i}: Success - {response}")
    except Exception as e:
        print(f"Request {i}: Error - {response}")
    
    # time.sleep(0.1)


In [ ]:
# try to make a lot of request in the same time -> still not work -> not reach the rate limit
import concurrent.futures

def make_request(i):
    try:
        response = client.text_classification(f"text {i}")
        return f"Request {i}: Success"
    except Exception as e:
        return f"Request {i}: Error - {e}"

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(make_request, i) for i in range(100)]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())